# Qwen Coder - Simple & Reliable\n\nUses Ollama's native OpenAI-compatible endpoint.

In [ ]:
!nvidia-smi
!curl -fsSL https://ollama.com/install.sh | sh
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb
print('✅ Installed')

In [ ]:
import subprocess, time, os
os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
!ollama pull qwen2.5-coder:7b
print('✅ Model ready')

In [ ]:
# Test Ollama's OpenAI endpoint directly
import requests
r = requests.post('http://localhost:11434/v1/chat/completions', json={
    'model': 'qwen2.5-coder:7b',
    'messages': [{'role': 'user', 'content': 'Say hello in one word'}]
})
print('Status:', r.status_code)
print('Response:', r.json())

In [ ]:
# Tunnel directly to Ollama's OpenAI endpoint
import subprocess, re
from IPython.display import display, HTML

tunnel = subprocess.Popen(
    ['cloudflared', 'tunnel', '--url', 'http://localhost:11434'],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)

for line in tunnel.stdout:
    print(line, end='')
    if 'trycloudflare.com' in line:
        m = re.search(r'https://[^\s]+\.trycloudflare\.com', line)
        if m:
            url = m.group()
            display(HTML(f'''<div style="background:linear-gradient(135deg,#667eea,#764ba2);padding:20px;border-radius:10px">
            <h2 style="color:white">🎉 Ready!</h2>
            <code style="color:white;font-size:16px">{url}/v1</code>
            <p style="color:#eee;margin-top:10px">OpenCode: /connect → Other → Base URL: {url}/v1</p>
            </div>'''))
            print(f'\n\n📋 Copy this: {url}/v1')
            break

for line in tunnel.stdout:
    print(line, end='')